No Place for Hate

We seek to find the correlation of the occurrence of Hate Crimes and:
* Education
* Poverty
* Unemployment Rates




In [1]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress


In [2]:
# import and merge data sets - Education, and Unemployment Rates by state
#read Education csv file and review the sample rows
edu_data = pd.read_csv("Resources/EducationReport.csv")
edu_data.head(3)

,Unnamed: 0,Total,Total.1,Total.2,Total.3,Total.4,Urban,Urban.1,Urban.2,Urban.3,Urban.4,Rural,Rural.1,Rural.2,Rural.3,Rural.4
0,Name,1970,1980,1990,2000,2014-2018,1970,1980,1990,2000,2014-2018,1970,1980,1990,2000,2014-2018
1,United States,10.70%,16.20%,20.30%,24.40%,31.50%,11.50%,17.50%,22.00%,26.20%,33.50%,6.70%,10.50%,12.30%,14.90%,19.60%
2,Alabama,7.80%,12.20%,15.70%,19.00%,24.90%,8.90%,13.70%,17.80%,21.70%,28.00%,5.30%,8.20%,9.80%,11.60%,15.20%


In [11]:
#create a new dataframe with only needed columns for years 2014-2018
edu_data_df = pd.DataFrame(edu_data)
edu_data_mycolm = edu_data_df[["Unnamed: 0", "Total.4", "Urban.4", "Rural.4"]]
edu_data_upd = edu_data_mycolm.rename(columns={"Unnamed: 0" : "State Name", "Total.4" : "Total Edu", "Urban.4" : "Total Edu Urban", "Rural.4" : "Total Edu Rural"})
edu_data_upd.head(5)

,State Name,Total Edu,Total Edu Urban,Total Edu Rural
0,Name,2014-2018,2014-2018,2014-2018
1,United States,31.50%,33.50%,19.60%
2,Alabama,24.90%,28.00%,15.20%
3,Alaska,29.20%,31.80%,23.90%
4,Arizona,28.90%,29.60%,15.80%


In [4]:
unempl_data = pd.read_csv("Resources/Unemployment.csv", encoding="ISO-8859-1")
unempl_data.head(5)

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"161,389,026","155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,216,627","2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0
2,1001,AL,"Autauga County, AL",2.0,2.0,1.0,"21,720","20,846",874,4.0,...,"26,196","25,261",935,3.6,"26,172","25,458",714,2.7,"59,338",119.0
3,1003,AL,"Baldwin County, AL",3.0,2.0,1.0,"69,533","66,971","2,562",3.7,...,"95,233","91,809","3,424",3.6,"97,328","94,675","2,653",2.7,"57,588",115.5
4,1005,AL,"Barbour County, AL",6.0,6.0,0.0,"11,373","10,748",625,5.5,...,"8,414","7,987",427,5.1,"8,537","8,213",324,3.8,"34,382",68.9


In [12]:
#filter data to get to the state level (the columns where rural urban code & metro 2013 columns are empty)
unempl_data['Rural_urban_continuum_code_2013'].replace('', np.nan, inplace=True)
unempl_state_interim = unempl_data[unempl_data['Rural_urban_continuum_code_2013'].isnull()]
unempl_state_interim
unempl_data_clean = unempl_state_interim[unempl_state_interim['Metro_2013']!=0]
unempl_data_clean.head(3)

,FIPStxt,Stabr,area_name,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,Civilian_labor_force_2000,Employed_2000,Unemployed_2000,Unemployment_rate_2000,...,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Civilian_labor_force_2019,Employed_2019,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
0,0,US,United States,NaN,NaN,NaN,"142,601,667","136,904,680","5,696,987",4.0,...,"161,389,026","155,102,319","6,286,707",3.9,"163,100,055","157,115,247","5,984,808",3.7,"61,937",NaN
1,1000,AL,Alabama,NaN,NaN,NaN,"2,133,223","2,035,594","97,629",4.6,...,"2,216,627","2,130,845","85,782",3.9,"2,241,747","2,174,483","67,264",3.0,"49,881",100.0
69,2000,AK,Alaska,NaN,NaN,NaN,"319,511","299,146","20,365",6.4,...,"352,662","329,901","22,761",6.5,"347,779","326,430","21,349",6.1,"74,912",100.0


In [18]:
#select columns needed for years 2014-2018
unempl_data_mycolm = unempl_data_clean[["Stabr", "area_name", "Civilian_labor_force_2018", "Employed_2018", "Unemployed_2018", "Unemployment_rate_2018"]]
unempl_data_mycolm

,Stabr,area_name,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018
0,US,United States,"161,389,026","155,102,319","6,286,707",3.9
1,AL,Alabama,"2,216,627","2,130,845","85,782",3.9
69,AK,Alaska,"352,662","329,901","22,761",6.5
102,AZ,Arizona,"3,422,583","3,260,518","162,065",4.7
118,AR,Arkansas,"1,353,305","1,303,995","49,310",3.6
194,CA,California,"19,280,821","18,460,725","820,096",4.3
253,CO,Colorado,"3,080,661","2,983,498","97,163",3.2
318,CT,Connecticut,"1,900,664","1,821,891","78,773",4.1
327,DE,Delaware,"481,773","463,542","18,231",3.8
331,DC,District of Columbia,"404,739","381,703","23,036",5.7


In [ ]:
# Merge the merged Demographic Data with Hate Crimes 358 by state

In [ ]:
# occurrence of Hate Crimes to Education (Bachelor's Degree vs. High School) by State

In [ ]:
# occurrence of Hate Crimes to Poverty Level by State

In [ ]:
# occurrence of Hate Crimes to Poverty Level by State

In [ ]:
# Clean FBI Data set - take 2014-2018 data and average hate crimes fbi